In [ ]:
!pip install bitsandbytes
!pip install transformers
!pip install peft
!pip install accelerate
!pip install einops
!pip install datasets

In [1]:
!export HF_DATASETS_CACHE="/cstor/mendeza/hf_test/"

In [1]:
CACHE_DIR="/cstor/mendeza/hf_test/"

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_id = 'mosaicml/mpt-7b'

model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    device_map='auto',
    trust_remote_code=True,
    cache_dir=CACHE_DIR
)

2023-06-05 23:23:04.843889: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 23:23:05.462309: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/lib/libibverbs:/container/ucx/lib:/container/ucx/lib64:/container/ompi/lib:/.singularity.d/libs
2023-06-05 23:23:05.462385: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; 

You are using config.init_device='cpu', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.57s/it]


In [4]:
model_size = sum(t.numel() for t in model.parameters())
print(f"Modelsize: {model_size/1000**2:.1f}M parameters")

Modelsize: 6649.3M parameters


In [5]:
# EleutherAI/gpt-neox-20b
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neox-20b',cache_dir=CACHE_DIR)
tokenizer.pad_token = tokenizer.eos_token

In [6]:
from datasets import set_caching_enabled
set_caching_enabled(False)

/tmp/ipykernel_28758/1910293094.py:2: FutureWarning: set_caching_enabled is deprecated and will be removed in the next major version of datasets. Use datasets.enable_caching() or datasets.disable_caching() instead. This function will be removed in a future version of datasets.
  set_caching_enabled(False)


In [15]:
from datasets import load_dataset
from pathlib import Path
dataset = load_dataset(path="wikitext", name="wikitext-2-v1",cache_dir=Path(CACHE_DIR),download_mode='force_redownload')

Dataset wikitext downloaded and prepared to /cstor/mendeza/hf_test/wikitext/wikitext-2-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 1029.28it/s]


In [16]:
# !rm -rf /home/mendeza/.cache/ -v

In [17]:
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                                  BertConfig, BertForMaskedLM, BertTokenizer,
                                  GPT2Config, GPT2LMHeadModel, GPT2Tokenizer,
                                  OpenAIGPTConfig, OpenAIGPTLMHeadModel, OpenAIGPTTokenizer,
                                  RobertaConfig, RobertaForMaskedLM, RobertaTokenizer,
                                  DistilBertConfig, DistilBertForMaskedLM, DistilBertTokenizer,TextDataset,DataCollatorForLanguageModeling)
from tqdm import tqdm, trange
import random
import numpy as np

In [19]:
dataset['train'][10]

{'text': ' The game \'s battle system , the <unk> system , is carried over directly from <unk> Chronicles . During missions , players select each unit using a top @-@ down perspective of the battlefield map : once a character is selected , the player moves the character around the battlefield in third @-@ person . A character can only act once per @-@ turn , but characters can be granted multiple turns at the expense of other characters \' turns . Each character has a field and distance of movement limited by their Action <unk> . Up to nine characters can be assigned to a single mission . During gameplay , characters will call out if something happens to them , such as their health points ( HP ) getting low or being knocked out by enemy attacks . Each character has specific " Potentials " , skills unique to each character . They are divided into " Personal Potential " , which are innate skills that remain unaltered unless otherwise dictated by the story and can either help or impede a 

In [20]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)
def tokenize_function(examples):
    output = tokenizer(examples['text'])
    # clm input could be much much longer than block_size
    return output
tokenized_datasets = dataset.map(
                tokenize_function,
                batched=True,
                remove_columns=["text"]
            )

In [22]:
tokenized_datasets['train'][1]

{'input_ids': [426, 657, 1278, 90, 5182, 28289, 868, 6490, 426, 2490],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [29]:
# train_sampler = RandomSampler(train_dataset) if local_rank == -1 else DistributedSampler(train_dataset)
num_train_epochs=1
train_batch_size=16
gradient_accumulation_steps = 1
learning_rate=5e-5
# num_train_epochs=1
# train_batch_size = 32
# eval_batch_size=32
gradient_accumulation_steps = 1
adam_epsilon=1e-8
warmup_steps=0
weight_decay=0.0
local_rank=-1
n_gpu=1
train_dataset=tokenized_datasets['train']
train_sampler = RandomSampler(train_dataset)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
# train_dataloader = DataLoader(dataset, collate_fn =data_collator ,sampler=train_sampler, batch_size=train_batch_size)
train_dataloader = DataLoader(train_dataset ,sampler=train_sampler, batch_size=train_batch_size)
t_total = len(dataset) // gradient_accumulation_steps * num_train_epochs

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total)

# Train!
# logger.info("***** Running training *****")
# logger.info("  Num examples = %d", len(dataset))
# logger.info("  Num Epochs = %d", num_train_epochs)
# logger.info("  Instantaneous batch size per GPU = %d", per_gpu_train_batch_size)
# logger.info("  Total train batch size (w. parallel, distributed & accumulation) = %d",
#                train_batch_size * gradient_accumulation_steps * (torch.distributed.get_world_size() if local_rank != -1 else 1))
# logger.info("  Gradient Accumulation steps = %d", gradient_accumulation_steps)
# logger.info("  Total optimization steps = %d", t_total)
# tb_writer = SummaryWriter()
global_step = 0
tr_loss, logging_loss = 0.0, 0.0
model.resize_token_embeddings(len(tokenizer))
model.zero_grad()
train_iterator = trange(int(num_train_epochs), desc="Epoch", disable=local_rank not in [-1, 0])

set_seed(0)
for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=local_rank not in [-1, 0])
    for step, batch in enumerate(epoch_iterator):
        # inputs, labels = mask_tokens(batch, tokenizer, None) if mlm else (batch, batch)
        # inputs = inputs.to(device)
        # labels = labels.to(device)
        print(batch)
        inputs, labels = format_batch(batch)
        model.train()

        outputs = model(inputs, masked_lm_labels=labels) if mlm else model(inputs, labels=labels)
        # print("outputs: ",outputs.keys())
        loss = outputs[0]  # model outputs are always tuple in transformers (see doc)
        if n_gpu > 1:
            loss = loss.mean()  # mean() to average on multi-gpu parallel training
        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps

        if fp16:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()
        loss.backward()
        tr_loss += loss.item()
        if (step + 1) % gradient_accumulation_steps == 0:
            if fp16:
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), max_grad_norm)
            else:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()  # Update learning rate schedule
            model.zero_grad()
            global_step += 1

        if local_rank in [-1, 0] and logging_steps > 0 and global_step % logging_steps == 0:
                # Log metrics
            if local_rank == -1 and evaluate_during_training:  # Only evaluate when single GPU otherwise metrics may not average well
                results = evaluate(model, tokenizer)
                for key, value in results.items():
                        tb_writer.add_scalar('eval_{}'.format(key), value, global_step)
                tb_writer.add_scalar('lr', scheduler.get_lr()[0], global_step)
                tb_writer.add_scalar('loss', (tr_loss - logging_loss)/logging_steps, global_step)
                logging_loss = tr_loss

            if local_rank in [-1, 0] and save_steps > 0 and global_step % save_steps == 0:
                checkpoint_prefix = 'checkpoint'
                # Save model checkpoint
                train_output_dir = os.path.join(output_dir, '{}-{}'.format(checkpoint_prefix, global_step))
                if not os.path.exists(train_output_dir):
                    os.makedirs(train_output_dir)
                model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
                model_to_save.save_pretrained(train_output_dir)
                # torch.save(os.path.join(train_output_dir, 'training_args.bin'))
                logger.info("Saving model checkpoint to %s", train_output_dir)

                # _rotate_checkpoints(args, checkpoint_prefix)

        if max_steps > 0 and global_step > max_steps:
            epoch_iterator.close()
            break
    if max_steps > 0 and global_step > max_steps:
        train_iterator.close()
        break


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: each element in list of batch should be of equal size